In [1]:
import pandas as pd

obj = pd.read_pickle(r'specific-token-100-200-activating-prompts.pkl')

In [3]:
!pip install openai
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 20.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 64.4 MB/s eta 0:00:00


In [17]:
import openai
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

openai.api_key = ""
context = "Please analyze the following list of tokens and identify whether they exhibit any semantic or syntactic themes. If so, respond 'Yes: ' followed by a list of no more than three themes in under ten words. If not, simply respond 'No'. (For example: 'Pizza, pasta, spaghetti, roads, bridges, ninety-seven' -> 'Yes: mostly italian food and infrastructure', 'quixotic, leftover, dice, _ly, 9178' -> 'No'). "
#context = "is there a rough, overarching theme tying most (i.e., >40%) of the following tokens together? We are using the term “theme” generously here — for example, if most of the outputs are numbers with no discernable pattern, then output “Yes: numbers”. This could also be a more broad one such as “religion”. There could also be two semantic meanings of a word that are both captured by the model — in this case, please provide both meanings.Please give your answer in the format 'Yes: (insert less than 4 word summary of concept)' or 'No', with no explanation."
def get_interpretability(act_prompts):
    interpretable_neurons = {}
    flag = False
    print(len(act_prompts.keys()))
    for neur in act_prompts.keys():
        print(neur)
        token_string = ": "
        for key in act_prompts[neur].keys():
            for token in act_prompts[neur][key]:
                token_string = token_string + ", " + token
                if len(encoding.encode(context + token_string)) > 4050:
                    flag = True
                    break
            if flag:
              flag = False
              break
        #print(len(encoding.encode(context + token_string)))
        response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo",
          messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": context + token_string},
            ]
        )
        interpretation = response['choices'][0]['message']['content']
        #print(token_string)
        #print(interpretation)
       # if "I'm sorry" in interpretation:
           # print(token_string)
        if "Yes" in interpretation:
           # print(interpretation[4:])
            interpretable_neurons[neur] = interpretation[4:]

    return interpretable_neurons

In [18]:
import pickle as pkl

interpretable_neurons = get_interpretability(obj)

#a = {'hello': 'world'}

with open('interpretable_neurons.pkl', 'wb') as handle:
    pkl.dump(interpretable_neurons, handle, protocol=pkl.HIGHEST_PROTOCOL)

with open('interpretable_neurons.pkl', 'rb') as handle:
    b = pkl.load(handle)

print(interpretable_neurons == b)

298
5
10
20
21
42
46
51
52
60
88
115
131
156
171
178
180
188
201
202
208
233
250
253
256
265
316
317
319
325
331
335
344
356
361
365
379
381
407
415
428
442
452
468
489
500
519
520
560
583
621
628
661
674
681
682
691
699
709
735
736
776
787
789
805
830
833
835
841
846
853
854
871
888
890
893
900
912
922
931
935
943
956
981
982
999
1012
1026
1030
1031
1032
1059
1060
1075
1096
1113
1127
1134
1198
1201
1202
1204
1214
1216
1236
1246
1256
1293
1302
1311
1313
1340
1359
1364
1366
1381
1387
1393
1419
1426
1432
1434
1439
1447
1470
1472
1473
1484
1496
1507
1515
1530
1539
1550
1559
1567
1569
1582
1583
1584
1594
1595
1615
1616
1625
1633
1642
1646
1647
1650
1657
1669
1673
1679
1703
1711
1734
1751
1754
1755
1758
1759
1766
1771
1785
1786
1787
1802
1808
1821
1828
1833
1837
1854
1855
1901
1910
1911
1919
1933
1936
1947
1951
1958
1968
1977
1985
1989
1999
2002
2010
2021
2029
2037
2052
2060
2064
2068
2070
2071
2073
2075
2089
2097
2102
2117
2119
2123
2126
2146
2149
2156
2166
2169
2212
2213
2218
2225
2228
22

In [20]:
len(interpretable_neurons.keys())

117

In [33]:
interpretable_neurons[361]

' Civil Rights, Protestants, Silk Road.'